# Is the NN enough to identify LRU or LFU indices?

In this notebook I check if the nn for my DQN even capable of doing its task?

In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import gym
import time
from e2 import CacheEnv
from tqdm import tqdm_notebook as tqdm

In [14]:
# env vars
EPS_LEN = 100
N_PAGES = 20
CACHE_LIMIT = 10 
env = CacheEnv(
        eps_len=EPS_LEN, 
        n_pages=N_PAGES, 
        limit=CACHE_LIMIT
        )

# dqn vars
# N_EPS = 60000
N_EPS = 1000
BATCH_SIZE = 32
LR_adam = 3e-4                   # learning rate for Adam
LR_sgd = 1e-3                   # learning rate for SGD
EPSILON = 0.9               # greedy policy
GAMMA = 0.9                 # reward discount
TARGET_REPLACE_ITER = 2000   # target update frequency
MEMORY_CAPACITY = 20000

s = env.reset()
N_ACTIONS = env.action_space_n
STATE_SHAPE = (CACHE_LIMIT, 2)
N_STATES = STATE_SHAPE[0]*STATE_SHAPE[1]

Cache limit: 10
Total Pages: 20


In [16]:
# Collect data
dataX = []
for _ in tqdm(range(N_EPS)):
    s = env.reset()
    dataX.append(s)

    while True:
        a = np.random.randint(0, N_ACTIONS)
        s, _, done, _ = env.step(a)
        dataX.append(s)
        
        if done:
            break

print(len(dataX))            


101000


In [17]:
def get_labels(dataX):
    dataYLU = []
    dataYRU = [] 
    for x in tqdm(dataX):
        lus = np.argmin(x[::2])
        rus = np.argmin(x[1::2])
        dataYLU.append(lus)
        dataYRU.append(rus)
    return dataYLU, dataYRU     
        
dataYLU, dataYRU = get_labels(dataX)

In [18]:
print(len(dataX))
print(len(dataYLU))
print(len(dataYRU))

101000
101000
101000


In [19]:
from sklearn.model_selection import train_test_split
X = dataX
Y = dataYLU
XTrain, XTest, yTrain, yTest = train_test_split(X, Y, test_size = 0.2)
print(len(X))
print(len(Y))
print(len(XTrain))
print(len(XTest))

101000
101000
80800
20200


In [20]:
from torch.utils.data import Dataset, DataLoader

class CacheDataset(Dataset):
    def __init__(self, data, targets, transform=None):
        self.transform = transform
        self.data = torch.Tensor(data)
        self.targets = torch.LongTensor(targets)

    def __getitem__(self, index):
        x = self.data[index]
        y = self.targets[index]
        return x, y

    def __len__(self):
        return self.data.shape[0]

In [21]:
train_dataset = CacheDataset(XTrain, yTrain)
test_dataset = CacheDataset(XTest, yTest)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

In [24]:
class Net(nn.Module):
    def __init__(self, ):
        super(Net, self).__init__()
        input_size = N_STATES
        h_dim = 50
        self.fc1 = nn.Linear(input_size, h_dim)
        self.fc2 = nn.Linear(h_dim, h_dim//4)
#         self.fc3 = nn.Linear(h_dim//4, h_dim)
        self.out = nn.Linear(h_dim//4, N_ACTIONS)

    def forward(self, x):
#         bs = x.shape[0]
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
#         x = F.relu(self.fc3(x))
        x = self.out(x)
#         return x
        return F.softmax(x, dim=0)    

In [23]:
model = Net().cuda()
LR_adam = 3e-3
optimizer = torch.optim.Adam(model.parameters(), lr=LR_adam)
# optimizer = torch.optim.SGD(model.parameters(), lr=LR_sgd)
# loss_func = nn.MSELoss()
criterion = nn.CrossEntropyLoss()

epochs = 100
for epoch in tqdm(range(epochs)):
    totalloss = []
    for i, (X, y) in enumerate(train_loader):
        X, y = X.cuda(), y.cuda()
        optimizer.zero_grad()
        out = model(X)
        loss = criterion(out, y)
        loss.backward()
        optimizer.step()
        with torch.no_grad():
            totalloss.append(loss.detach().cpu().numpy())
    totalloss = np.array(totalloss).mean()
    print(f"Epoch: [{epoch}] | Loss: {totalloss}")               

Epoch: [0] | Loss: 2.259532928466797
Epoch: [1] | Loss: 2.220102310180664
Epoch: [2] | Loss: 2.2002053260803223
Epoch: [3] | Loss: 2.1833508014678955
Epoch: [4] | Loss: 2.1603164672851562
Epoch: [5] | Loss: 2.128648042678833
Epoch: [6] | Loss: 2.093916416168213
Epoch: [7] | Loss: 2.0518012046813965
Epoch: [8] | Loss: 2.046609401702881
Epoch: [9] | Loss: 2.0448379516601562
Epoch: [10] | Loss: 2.043445587158203
Epoch: [11] | Loss: 2.0426876544952393
Epoch: [12] | Loss: 2.0422677993774414
Epoch: [13] | Loss: 2.041914939880371
Epoch: [14] | Loss: 2.0413691997528076
Epoch: [15] | Loss: 2.0413410663604736
Epoch: [16] | Loss: 2.040956497192383
Epoch: [17] | Loss: 2.04116153717041
Epoch: [18] | Loss: 2.040792942047119
Epoch: [19] | Loss: 2.0409271717071533
Epoch: [20] | Loss: 2.0407297611236572
Epoch: [21] | Loss: 2.040729522705078
Epoch: [22] | Loss: 2.040194272994995
Epoch: [23] | Loss: 2.040257215499878
Epoch: [24] | Loss: 2.040501356124878
Epoch: [25] | Loss: 2.040241241455078
Epoch: [26] 

In [29]:
model = Net().cuda()
LR_adam = 3e-3
optimizer = torch.optim.Adam(model.parameters(), lr=LR_adam)
# LR_sgd = 1
# optimizer = torch.optim.SGD(model.parameters(), lr=LR_sgd)
# loss_func = nn.MSELoss()
criterion = nn.CrossEntropyLoss()

epochs = 100
for epoch in tqdm(range(epochs)):
    totalloss = []
    for i, (X, y) in enumerate(train_loader):
        X, y = X.cuda(), y.cuda()
        optimizer.zero_grad()
        out = model(X)
        loss = criterion(out, y)
        loss.backward()
        optimizer.step()
        with torch.no_grad():
            totalloss.append(loss.detach().cpu().numpy())
    totalloss = np.array(totalloss).mean()
    print(f"Epoch: [{epoch}] | Loss: {totalloss}")               

Epoch: [0] | Loss: 2.2546284198760986
Epoch: [1] | Loss: 2.1946310997009277
Epoch: [2] | Loss: 2.1658849716186523
Epoch: [3] | Loss: 2.1551766395568848
Epoch: [4] | Loss: 2.1485772132873535
Epoch: [5] | Loss: 2.1315369606018066
Epoch: [6] | Loss: 2.1016390323638916
Epoch: [7] | Loss: 2.0723493099212646
Epoch: [8] | Loss: 2.0560410022735596
Epoch: [9] | Loss: 2.0498576164245605
Epoch: [10] | Loss: 2.0476937294006348
Epoch: [11] | Loss: 2.0468590259552
Epoch: [12] | Loss: 2.0463180541992188
Epoch: [13] | Loss: 2.0460495948791504
Epoch: [14] | Loss: 2.045947790145874
Epoch: [15] | Loss: 2.045693874359131
Epoch: [16] | Loss: 2.0456440448760986
Epoch: [17] | Loss: 2.045503616333008
Epoch: [18] | Loss: 2.0453832149505615
Epoch: [19] | Loss: 2.0452992916107178
Epoch: [20] | Loss: 2.0451860427856445
Epoch: [21] | Loss: 2.045102596282959
Epoch: [22] | Loss: 2.044992208480835
Epoch: [23] | Loss: 2.044844627380371
Epoch: [24] | Loss: 2.044734239578247
Epoch: [25] | Loss: 2.0446269512176514
Epoch:

KeyboardInterrupt: 